In [ ]:
!pip install torchmetrics
!pip install piqa
from piqa import MS_SSIM
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from torchmetrics import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from torchvision.io import read_image
from typing import Tuple,List,Callable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# use CUDA processors if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
def process_mat(directory: str, train: bool, test_files: list) -> Tuple[torch.Tensor, int, list, list]:
    # prepare feature data and image paths
    image_paths = []
    digits = []
    raw_data = None
    clean = f'{directory}/clean_split'
    for filename in os.listdir(clean): #iterate through the files in directory
        file = os.path.join(clean, filename)
        # checking if it is a file
        if os.path.isfile(file):
            if train:
                id = 'rain.pt'
            else:
                id = 'test.pt'
            if (filename[-7:] == id):
                if (id ==  'rain.pt') or (id == 'test.pt' and ((test_files == None) or (filename in test_files))):
                    #print(filename)
                    data = torch.load(file)
                    features = data[0]
                    labels = data[1]
                    if raw_data is None:
                        raw_data = features
                    else:
                        try:
                            raw_data = np.append(raw_data, features, axis=0)
                        except:
                            print(f"IGNORED: {filename}")
                            continue
                    for i, y in enumerate(labels):
                        if not os.path.isfile(y[0]):
                            #print(f"INGORED IMAGE: {y[0]}")
                            index = i - len(labels)
                            #print(labels[index][0]==y[0])
                            #print(raw_data.shape)
                            raw_data = np.delete(raw_data, index, 0)
                            continue
                        else:
                            image_paths.append(y[0])
                            digits.append(y[1])
                else:
                    continue
            else:
                continue
        else:
            raise Exception("this is not a file")
    #print(type(raw_data))
    raw_data = torch.tensor(raw_data.astype(np.float32)).cuda()
    num_features = raw_data.shape[1]
    #print(len(image_paths))
    return raw_data, num_features, image_paths, digits

In [ ]:
# define data transformations
def feature_transform(x: torch.Tensor):
    # standardizes the features of a given data point
    mean = x.mean()
    std = x.std()
    return x.sub(mean).div(std)

def img_label_transform(y: str):
    # transforms an image path to a usable tensor
    image = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    convert_tensor = A.Compose([A.ToFloat(max_value=255), ToTensorV2()])
    try:
        thingy = convert_tensor(image=image)['image']
    except Exception as e:
        print(e)
        raise Exception(f"Image not found: {y}")
    return thingy.cuda()

def digit_label_transform(y: str):
    return int(y)

# create train and test Datasets and DataLoaders
class ReconstructionData(Dataset):
    def __init__(self, raw_features: torch.Tensor, labels: list, feature_transform, label_transform: None):
        self.features = raw_features
        self.labels = labels
        self.feature_transform = feature_transform
        self.label_transform = label_transform


    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        data = self.feature_transform(self.features[idx])
        label = self.label_transform(self.labels[idx])
        return data, label


def prep_data(train_features: torch.Tensor, train_labels: list, test_features: torch.Tensor, test_labels: list, label_transform, batch_size:int) -> Tuple[DataLoader, DataLoader]:
    # create train and test dataloaders
    train_data = ReconstructionData(train_features, train_labels, feature_transform, label_transform)
    test_data = ReconstructionData(test_features, test_labels, feature_transform, label_transform)
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    return train_dataloader,test_dataloader

In [ ]:
class ImgReconstructNN(nn.Module):
    def __init__(self, num_features:int):
        super(ImgReconstructNN, self).__init__()
        self.linear_relu_deconv_stack = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.PReLU(),
            nn.Linear(512, 2048),
            nn.PReLU(),
            nn.Linear(2048, 28*28),
            nn.Sigmoid(),
            nn.Unflatten(1, (1, 28, 28))
        )
    def forward(self, x):
        logits = self.linear_relu_deconv_stack(x)
        return logits

class ClassifyNN(nn.Module):
    # INCOMPLETE
    # TO DO
    def __init__(self, num_features:int):
        super(ImgReconstructNN, self).__init__()
        self.linear_relu_deconv_stack = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.PReLU(),
            nn.Linear(512, 2048),
            nn.PReLU(),
            nn.Linear(2048, 28*28),
            nn.Sigmoid(),
            nn.Unflatten(1, (1, 28, 28))
        )
    def forward(self, x):
        logits = self.linear_relu_deconv_stack(x)
        return logits

In [ ]:
def create_model(num_features:int) -> nn.Module:
  model = ImgReconstructNN(num_features).to(device)
  print(model)
  return model

In [ ]:
# Training loop
def train_loop(dataloader: DataLoader, model: nn.Module, loss_fn, optimizer, print_loss: bool, loss_list: list) -> list:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= num_batches
    loss_list.append(train_loss)
    if print_loss:  
        print(f"Avg batch loss: {train_loss:>8f}")
    return loss_list

def test_loop(dataloader, model, loss_fn):
    print("Running Test Loop")
    num_batches = len(dataloader)
    test_loss = 0
    test_psnr = 0
    test_ssim = 0
    psnr = PeakSignalNoiseRatio().to(device)
    ssim = StructuralSimilarityIndexMeasure().to(device)

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):       
            print(f"batch: {batch+1}")     
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_ssim += ssim(pred, y).item()
            test_psnr += psnr(pred, y).item()

            # display 1 original and predicted image from each batch
            transform = transforms.ToPILImage()
            orig_data = y[0]
            orig_img = transform(orig_data)
            pred_data = pred[0]
            pred_img = transform(pred_data)
            print("original:")
            display(orig_img)
            print("predicted:")
            display(pred_img)
            print()

    test_loss /= num_batches
    test_psnr /= num_batches
    test_ssim /= num_batches
    print(f"Avg batch loss: {test_loss:>8f}")
    print(f"Avg batch PSNR: {test_psnr:>8f}")
    print(f"Avg batch SSIM: {test_ssim:>8f}")
    return model, test_loss, test_psnr, test_ssim

In [ ]:
def train_model(train_dataloader, model, loss_fn, optimizer, epochs):
  losses = []
  for t in range(epochs):
      print_loss = False
      if (t % 8 == 7) or (t==0):
        print_loss = True
        print("-------------------------------")
        print(f"Epoch {t+1}")
      losses = train_loop(train_dataloader, model, loss_fn, optimizer, print_loss, losses)
  print("Done!")
  return losses

In [ ]:
# Validation Results:
#  32-epoch loss: 0.014299
#  48-epoch loss: 0.013660 
#  64-epoch loss: 0.013298
#  80-epoch loss: 0.013655 
#  96-epoch loss: 0.013975
# 128-epoch loss: 0.014216
# 192-epoch loss: 0.015130

In [ ]:
def plot_loss(losses: list) -> None:
    losses = np.array(torch.tensor(losses).cpu())
    plt.plot(losses, color='red')
    plt.ylabel('Average Batch Loss') #set the label for y axis
    plt.xlabel('Epoch') #set the label for x-axis
    plt.title("Loss over Training Epochs") #set the title of the graph
    plt.show() #display the graph

In [ ]:
def test_run(directory: str, test_files, batch_size, loss_fn, learning_rate, epochs: int):
    #print("PART 1")
    train_data, num_features, train_image_paths, _ = process_mat(directory, True, test_files)
    #print("PART 2")
    test_data, num_features, test_image_paths, _ = process_mat(directory, False, test_files)
    #print("PART 3")
    train_dataloader, test_dataloader = prep_data(train_data, train_image_paths, test_data, test_image_paths, img_label_transform, batch_size)
    #print("PART 4")
    model = create_model(num_features)
    filename = f'{directory}/model.pt'
    model.load_state_dict(torch.load(filename))
    print(test_files[0])
    #print("PART 5")
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    #print("PART 6")
    #train_loss = train_model(train_dataloader, model, loss_fn, optimizer, epochs)
    #print("PART 7")
    #plot_loss(train_loss)
    #print("PART 8")
    model, test_loss, test_psnr, test_ssim = test_loop(test_dataloader, model, loss_fn)
    return model, test_loss, test_psnr, test_ssim

In [ ]:
l1 = nn.L1Loss()
class L1_SSIM_loss(MS_SSIM):
    def forward(self, x, y):
        return (0.5 * (1. - super().forward(x, y))) + (0.5 * l1.forward(x, y))

criterion = L1_SSIM_loss(window_size=2, n_channels=1).cuda()

In [ ]:
# prepare directories
main_dir = '/content/drive/MyDrive/superresolution/nonlinear_classification'
data_dir = f'{main_dir}/data/features/EXP_by_crystal'
image_dir = f'{main_dir}/data/digit_image'

# hyper-parameters:
batch_size = 50
learning_rate = 1e-3
epochs = 64
loss_fn = criterion # L1_SSIM_loss(window_size=4.3, n_channels=1).cuda()

# sample_dir = f'{data_dir}/original_EXP_data/with_noise/ChangeCrystalPosition_DATA/shiftCrystal40mm_SNR17(s10)_1-200samples_40LG_10loop_Oct12'
# run(sample_dir, test_size, batch_size, loss_fn, learning_rate, epochs)

In [ ]:
# 64 Epoch piqa
# Avg batch loss: 0.084998
# Avg batch PSNR: 15.341266
# Avg batch SSIM: 0.573755

# 128 Epoch piqa
# Avg batch loss: 0.063156
# Avg batch PSNR: 16.070961
# Avg batch SSIM: 0.636266

# 256 epoch 1/2 SSIM 1/2 MAE
# Avg batch loss: 0.057746
# Avg batch PSNR: 17.995046
# Avg batch SSIM: 0.764595

In [ ]:
sample_dir = f'{data_dir}/1f'
sample_clean = f'{sample_dir}/clean_split'
for filename in os.listdir(sample_clean): #iterate through the files in directory
    file = os.path.join(sample_clean, filename)
    # checking if it is a file
    if os.path.isfile(file):
        if (filename[-7:] == 'test.pt'):
            try:
                model, test_loss, test_psnr, test_ssim = test_run(sample_dir, [filename], batch_size, loss_fn, learning_rate, epochs)
            except:
                continue
        else:
            continue
    else:
        print(file)
        raise Exception("this is not a file")

IGNORED: 200-400sample_40LG_area40_amp100_pixel400_Aug28_10loop_10cluster_train.pt
ImgReconstructNN(
  (linear_relu_deconv_stack): Sequential(
    (0): Linear(in_features=40, out_features=512, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=512, out_features=2048, bias=True)
    (3): PReLU(num_parameters=1)
    (4): Linear(in_features=2048, out_features=784, bias=True)
    (5): Sigmoid()
    (6): Unflatten(dim=1, unflattened_size=(1, 28, 28))
  )
)
1-200sample_40LG_area40_amp100_pixel400_Aug9_10loop_10cluster_test.pt
Running Test Loop
batch: 1
original:


predicted:



batch: 2
original:


predicted:



batch: 3
original:


...


batch: 23
original:


predicted:



batch: 24
original:


predicted:



Avg batch loss: 0.129037
Avg batch PSNR: 14.727038
Avg batch SSIM: 0.481658


In [ ]:
sample_dir = f'{data_dir}/Crystal0.8f'
sample_clean = f'{sample_dir}/clean_split'
for filename in os.listdir(sample_clean): #iterate through the files in directory
    file = os.path.join(sample_clean, filename)
    # checking if it is a file
    if os.path.isfile(file):
        if (filename[-7:] == 'test.pt'):
            try:
                model, test_loss, test_psnr, test_ssim = test_run(sample_dir, [filename], batch_size, loss_fn, learning_rate, epochs)
            except:
                continue
        else:
            continue
    else:
        print(file)
        raise Exception("this is not a file")

ImgReconstructNN(
  (linear_relu_deconv_stack): Sequential(
    (0): Linear(in_features=40, out_features=512, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=512, out_features=2048, bias=True)
    (3): PReLU(num_parameters=1)
    (4): Linear(in_features=2048, out_features=784, bias=True)
    (5): Sigmoid()
    (6): Unflatten(dim=1, unflattened_size=(1, 28, 28))
  )
)
shiftCystal_0.8f_200-400_40LG_10loop_Aug23_test.pt
Running Test Loop
batch: 1
original:


predicted:



batch: 2
original:


predicted:



batch: 3
original:


...


batch: 7
original:


predicted:



batch: 8
original:


predicted:



Avg batch loss: 0.074938
Avg batch PSNR: 16.645025
Avg batch SSIM: 0.742497


In [ ]:
sample_dir = f'{data_dir}/Crystal27mm'
model, train_loss, test_loss, test_psnr, test_ssim = run(sample_dir, None, batch_size, loss_fn, learning_rate, epochs)
filename = f'{sample_dir}/model.pt'
torch.save(model.state_dict(), filename)
filename = f'{sample_dir}/train_loss.pt'
torch.save(train_loss, filename)
filename = f'{sample_dir}/test_loss.pt'
torch.save(test_loss, filename)
filename = f'{sample_dir}/test_psnr.pt'
torch.save(test_psnr, filename)
filename = f'{sample_dir}/test_ssim.pt'
torch.save(test_ssim, filename)

PART 1
shiftCrystal27mm_SNR15(s10)_1-200samples_40LG_10loop_Sep13_train.pt
shiftCrystal27mm_SNR10(s10)_1-200samples_40LG_10loop_Sep9_train.pt
shiftCrystal27mm_SNR17(s10)_1-200samples_40LG_10loop_Sep15_train.pt
shiftCrystal27mm_1-200samples_40LG_10loop_Sep3_train.pt
shiftCrystal27mm_1-600samples_40LG_10loop_Sep7_train.pt
<class 'numpy.ndarray'>
11200
PART 2
shiftCrystal27mm_SNR15(s10)_1-200samples_40LG_10loop_Sep13_test.pt
shiftCrystal27mm_SNR10(s10)_1-200samples_40LG_10loop_Sep9_test.pt
shiftCrystal27mm_SNR17(s10)_1-200samples_40LG_10loop_Sep15_test.pt
shiftCrystal27mm_1-200samples_40LG_10loop_Sep3_test.pt
shiftCrystal27mm_1-600samples_40LG_10loop_Sep7_test.pt
<class 'numpy.ndarray'>
2800
PART 3
PART 4
ImgReconstructNN(
  (linear_relu_deconv_stack): Sequential(
    (0): Linear(in_features=41, out_features=512, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=512, out_features=2048, bias=True)
    (3): PReLU(num_parameters=1)
    (4): Linear(in_features=2048, out_

KeyboardInterrupt: ignored

In [ ]:
sample_dir = f'{data_dir}/Crystal40mm'
model, train_loss, test_loss, test_psnr, test_ssim = run(sample_dir, None, batch_size, loss_fn, learning_rate, epochs)
filename = f'{sample_dir}/model.pt'
torch.save(model.state_dict(), filename)
filename = f'{sample_dir}/train_loss.pt'
torch.save(train_loss, filename)
filename = f'{sample_dir}/test_loss.pt'
torch.save(test_loss, filename)
filename = f'{sample_dir}/test_psnr.pt'
torch.save(test_psnr, filename)
filename = f'{sample_dir}/test_ssim.pt'
torch.save(test_ssim, filename)

In [19]:
# !pip install torchmetrics
# !pip install piqa
from piqa import MS_SSIM
from torchmetrics import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
import cv2
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from typing import Tuple, Callable


from google.colab import drive
drive.mount('/content/drive')

# use CUDA processors if available
vers = torch.version.cuda
print(vers)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


# prepare directories
main_dir = '/content/drive/MyDrive/superresolution/nonlinear_classification'
data_dir = os.path.join(main_dir, 'data', 'features', 'SIM_by_crystal')
image_dir = os.path.join(main_dir, 'data', 'digit_image')

# hyper-parameters:
batch_size = 50
learning_rate = 1e-3
epochs = 512


def adjust_img_path(path: str) -> str:
    # modify image path to work in desktop version of google drive
    return os.path.join(main_dir, *(path.split('/')[-4:]))


def process_mat(directory: str, train: bool, test_files: list) -> Tuple[torch.Tensor, int, list, list]:
    # prepare feature data and image paths
    image_paths = []
    digits = []
    raw_data = None
    clean = os.path.join(directory, 'clean_split')
    # iterate through the files in directory
    for filename in os.listdir(clean):
        file = os.path.join(clean, filename)
        # checking if it is a file
        if os.path.isfile(file):
            if train:
                id = 'rain.pt'
            else:
                id = 'test.pt'
            if (filename[-7:] == id):
                if (id == 'rain.pt') or (id == 'test.pt' and ((test_files == None) or (filename in test_files))):
                    # print(filename)
                    data = torch.load(file)
                    features = data[0]
                    labels = data[1]
                    if raw_data is None:
                        raw_data = features
                    else:
                        try:
                            raw_data = np.append(raw_data, features, axis=0)
                        except:
                            # print(f"IGNORED DATA: {filename}")
                            continue
                    for i, y in enumerate(labels):
                        img_path = adjust_img_path(y[0])
                        if not os.path.isfile(img_path):
                            # print(f"INGORED IMAGE: {img_path}")
                            index = i - len(labels)
                            raw_data = np.delete(raw_data, index, 0)
                            continue
                        else:
                            image_paths.append(img_path)
                            digits.append(y[1])
                else:
                    continue
            else:
                continue
        else:
            raise Exception("this is not a file")
    raw_data = torch.tensor(raw_data.astype(np.float32)).cuda()
    num_features = raw_data.shape[1]
    return raw_data, num_features, image_paths, digits

# define data transformations


def feature_transform(x: torch.Tensor):
    # standardizes the features of a given data point
    mean = x.mean()
    std = x.std()
    return x.sub(mean).div(std)


convert_tensor = A.Compose([A.ToFloat(max_value=255), ToTensorV2()])


def img_label_transform(y: str):
    # transforms an image path to a usable tensor
    image = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    try:
        label = convert_tensor(image=image)['image']
    except Exception as e:
        print(e)
        raise Exception(f"Image not found: {y}")
    return label.cuda()


def digit_label_transform(y: str):
    return int(y)

# create train and test Datasets and DataLoaders


class ReconstructionData(Dataset):
    def __init__(self, raw_features: torch.Tensor, labels: list, feature_transform, label_transform: None):
        self.features = raw_features
        self.labels = labels
        self.feature_transform = feature_transform
        self.label_transform = label_transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        data = self.feature_transform(self.features[idx])
        label = self.label_transform(self.labels[idx])
        return data, label


def prep_data(train_features: torch.Tensor, train_labels: list, test_features: torch.Tensor, test_labels: list, label_transform, batch_size: int) -> Tuple[DataLoader, DataLoader]:
    # create train and test dataloaders
    train_data = ReconstructionData(
        train_features, train_labels, feature_transform, label_transform)
    test_data = ReconstructionData(
        test_features, test_labels, feature_transform, label_transform)
    train_dataloader = DataLoader(
        train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(
        test_data, batch_size=batch_size, shuffle=True)
    return train_dataloader, test_dataloader


class ImgReconstructNN(nn.Module):
    def __init__(self, num_features: int):
        super(ImgReconstructNN, self).__init__()
        self.linear_relu_deconv_stack = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.PReLU(),
            nn.Linear(512, 2048),
            nn.PReLU(),
            nn.Linear(2048, 28*28),
            nn.Sigmoid(),
            nn.Unflatten(1, (1, 28, 28))
        )

    def forward(self, x):
        logits = self.linear_relu_deconv_stack(x)
        return logits


class ClassifyNN(nn.Module):
    # INCOMPLETE
    # TO DO
    def __init__(self, num_features: int):
        super(ImgReconstructNN, self).__init__()
        self.linear_relu_deconv_stack = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.PReLU(),
            nn.Linear(512, 2048),
            nn.PReLU(),
            nn.Linear(2048, 28*28),
            nn.Sigmoid(),
            nn.Unflatten(1, (1, 28, 28))
        )

    def forward(self, x):
        logits = self.linear_relu_deconv_stack(x)
        return logits


def create_model(num_features: int, verbose: bool) -> nn.Module:
    model = ImgReconstructNN(num_features).to(device)
    if verbose:
        print(model)
    return model

# Training loop


def train_loop(dataloader: DataLoader, model: nn.Module, loss_fn, optimizer, print_loss: bool, loss_list: list) -> list:
    num_batches = len(dataloader)
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= num_batches
    loss_list.append(train_loss)
    if print_loss:
        print(f"Avg batch loss: {train_loss:>8f}")
    return loss_list


def test_loop(dataloader, model, loss_fn, verbose: bool):
    print("Running Test Loop")
    num_batches = len(dataloader)
    test_loss = 0
    test_psnr = 0
    test_ssim = 0
    psnr = PeakSignalNoiseRatio().to(device)
    ssim = StructuralSimilarityIndexMeasure().to(device)

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            # print(f"batch: {batch+1}")
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            test_ssim += ssim(pred, y).item()
            test_psnr += psnr(pred, y).item()

            # # display 1 original and predicted image from each batch
            # transform = transforms.ToPILImage()
            # orig_data = y[0]
            # orig_img = transform(orig_data)
            # pred_data = pred[0]
            # pred_img = transform(pred_data)
            # print("original:")
            # orig_img.show()
            # print("predicted:")
            # pred_img.show()
            # print()

    test_loss /= num_batches
    test_psnr /= num_batches
    test_ssim /= num_batches
    if verbose:
        print(f"Avg batch loss: {test_loss:>8f}")
        print(f"Avg batch PSNR: {test_psnr:>8f}")
        print(f"Avg batch SSIM: {test_ssim:>8f}")
    return model, test_loss, test_psnr, test_ssim


def train_model(train_dataloader, model, loss_fn, optimizer, epochs):
    losses = []
    for t in range(epochs):
        print_loss = False
        if (t % 16 == 15) or (t == 0):
            print_loss = True
            print("-------------------------------")
            print(f"Epoch {t+1}")
        losses = train_loop(train_dataloader, model, loss_fn,
                            optimizer, print_loss, losses)
    print("Done!")
    return losses


def plot_loss(losses: list) -> None:
    losses = np.array(torch.tensor(losses).cpu())
    plt.plot(losses, color='red')
    plt.ylabel('Average Batch Loss')  # set the label for y axis
    plt.xlabel('Epoch')  # set the label for x-axis
    plt.title("Loss over Training Epochs")  # set the title of the graph
    plt.show()  # display the graph


def run(directory: str, test_files, batch_size, loss_fn, learning_rate, epochs: int):
    print("PART 1")
    train_data, num_features, train_image_paths, _ = process_mat(
        directory, True, None)
    print("PART 2")
    test_data, num_features, test_image_paths, _ = process_mat(
        directory, False, None)
    print("PART 3")
    train_dataloader, test_dataloader = prep_data(
        train_data, train_image_paths, test_data, test_image_paths, img_label_transform, batch_size)
    print("PART 4")
    model = create_model(num_features, True)
    print("PART 5")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print("PART 6")
    train_loss = train_model(train_dataloader, model,
                             loss_fn, optimizer, epochs)
    print("PART 7")
    # plot_loss(train_loss)
    print("PART 8")
    model, test_loss, test_psnr, test_ssim = test_loop(
        test_dataloader, model, loss_fn, True)
    return model, train_loss, test_loss, test_psnr, test_ssim


l1 = nn.L1Loss()


class L1_SSIM_loss(MS_SSIM):
    def forward(self, x, y):
        return (0.5 * (1. - super().forward(x, y))) + (0.5 * l1.forward(x, y))


loss_fn = L1_SSIM_loss(window_size=2, n_channels=1).cuda()
# sample_dir = f'{data_dir}/original_EXP_data/with_noise/ChangeCrystalPosition_DATA/shiftCrystal40mm_SNR17(s10)_1-200samples_40LG_10loop_Oct12'
# run(sample_dir, test_size, batch_size, loss_fn, learning_rate, epochs)

# 64 Epoch piqa
# Avg batch loss: 0.084998
# Avg batch PSNR: 15.341266
# Avg batch SSIM: 0.573755

# 128 Epoch piqa
# Avg batch loss: 0.063156
# Avg batch PSNR: 16.070961
# Avg batch SSIM: 0.636266

# 256 epoch 1/2 SSIM 1/2 MAE
# Avg batch loss: 0.057746
# Avg batch PSNR: 17.995046
# Avg batch SSIM: 0.764595
crystals = ['27mm']
# for crystal in crystals:
#     sample_dir = os.path.join(data_dir, crystal)
#     model, train_loss, test_loss, test_psnr, test_ssim = run(sample_dir, None, batch_size, loss_fn, learning_rate, epochs)
#     filename = os.path.join(sample_dir, 'model.pt')
#     torch.save(model.state_dict(), filename)
#     filename = os.path.join(sample_dir, 'train_loss.pt')
#     torch.save(train_loss, filename)
#     filename = os.path.join(sample_dir, 'test_loss.pt')
#     torch.save(test_loss, filename)
#     filename = os.path.join(sample_dir, 'test_psnr.pt')
#     torch.save(test_psnr, filename)
#     filename = os.path.join(sample_dir, 'test_ssim.pt')
#     torch.save(test_ssim, filename)


def test_run(directory: str, test_files, batch_size, loss_fn, learning_rate, epochs: int):
    train_data, _, train_image_paths, _ = process_mat(
        directory, True, test_files)
    test_data, num_features, test_image_paths, _ = process_mat(
        directory, False, test_files)
    _, test_dataloader = prep_data(
        train_data, train_image_paths, test_data, test_image_paths, img_label_transform, batch_size)
    model = create_model(106, False)
    filename = os.path.join(directory, 'model.pt')
    model.load_state_dict(torch.load(filename))
    model, test_loss, test_psnr, test_ssim = test_loop(
        test_dataloader, model, loss_fn, False)
    return model, test_loss, test_psnr, test_ssim


results = {}
results['crystal_position'] = []
results['filename'] = []
results['test_loss'] = []
results['test_psnr'] = []
results['test_ssim'] = []

for crystal in crystals:
    sample_dir = os.path.join(data_dir, crystal)
    sample_clean = os.path.join(sample_dir, 'clean_split')
    # iterate through the files in directory
    for filename in os.listdir(sample_clean):
        file = os.path.join(sample_clean, filename)
        # checking if it is a file
        if os.path.isfile(file):
            if (filename[-7:] == 'test.pt'):
                try:
                    model, test_loss, test_psnr, test_ssim = test_run(
                        sample_dir, [filename], batch_size, loss_fn, learning_rate, epochs)
                    results['crystal_position'].append(crystal)
                    results['filename'].append(filename)
                    results['test_loss'].append(test_loss)
                    results['test_psnr'].append(test_psnr)
                    results['test_ssim'].append(test_ssim)
                except Exception as e:
                    print(e)
                    continue
            else:
                continue
        else:
            print(file)
            raise Exception("this is not a file")

results = pd.DataFrame.from_dict(results)
filename = os.path.join(main_dir, 'data', 'sim_results.csv')
results.to_csv(filename, index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
11.6
Using cuda device
Running Test Loop
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x112 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x40 and 106x512)
Running Test Loop
mat1 and mat2 shapes cannot be multiplied (50x49 and 106x512)
